# 🌌 N-Body Simulation

#### Pragun Nepal  
#### June 21, 2025

---

This notebook contains the code for running the N-body Simulations (with and without axes). The full video of the simulations have been uploaded in my YouTube channel - https://www.youtube.com/@PragunNepal

All the data for this simulation have been obatined from the coupled N-body and FoF-Halo-Finder Repositories by Dr. Rajesh Mondal (GitHub : rajeshmondal18 - https://github.com/rajeshmondal18 )

---

## 📚 Table of Contents

1. [Setting up ffmpeg for Jupyter Lab](#Setting_up_ffmpeg_for_Jupyter_Lab)
2. [Code for N-body Simulation](#Code_for_N-body_Simulation)
3. [Code for N-body Simulation (with axes)](#Code_for_N-body_Simulation_axes)
4. [Conclusion](#conclusion)

---

## 🔭 <a name="Setting_up_ffmpeg_for_Jupyter_Lab"></a>1. Setting up ffmpeg for Jupyter Lab



```python
import os
import matplotlib as mpl

#Enter the Full Fath of the ffmeg library - It may be different for you
ffmpeg_path = "/opt/homebrew/bin/ffmpeg"

#Prepend its directory to PATH for this session
os.environ["PATH"] = os.path.dirname(ffmpeg_path) + os.pathsep + os.environ["PATH"]

#Tell Matplotlib exactly where to find it
mpl.rcParams["animation.ffmpeg_path"] = ffmpeg_path

#Verification
import shutil
print("→ PATH is now:", os.environ["PATH"].split(os.pathsep)[:3], "…")
print("→ which(ffmpeg):", shutil.which("ffmpeg"))
print("→ rcParams ffmpeg_path:", mpl.rcParams["animation.ffmpeg_path"])
```



If ffmpeg is not installed in your PC. You can do so by visiting the website - https://ffmpeg.org/download.html



---

## ⚙️ <a name="Code_for_N-body_Simulation"></a>2. Code for N-body Simulation (without axes)

```python
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

#Extracting Redshift Value from FileName

def extract_redshift(fn):
    base = os.path.splitext(os.path.basename(fn))[0]
    parts = base.split("_")
    return float(parts[1] + "." + parts[2])

#Load & sort data

files = sorted(glob.glob("p_*.txt"), key=extract_redshift, reverse=True)

#If the files are not in the same directory, you can add its path
""" 
nbody_path = "your file path here"
nbody_files = sorted(
    glob.glob(os.path.join(nbody_path, "nbody_*.txt")),
    key=extract_redshift_halo,
    reverse=True 
"""

redshifts = [extract_redshift(f) for f in files]
raw_frames = [np.loadtxt(f).reshape(-1,3) for f in files]

#Interpolate data & redshifts

frames_data, frames_z = [], []
for (a, za), (b, zb) in zip(zip(raw_frames[:-1], redshifts[:-1]), zip(raw_frames[1:], redshifts[1:])):
    frames_data.append(a); frames_z.append(za)
    frames_data.append((a + b)/2); frames_z.append((za + zb)/2)
frames_data.append(raw_frames[-1]); frames_z.append(redshifts[-1])

#Animation parameters

fps = 30
rotation_speed = 0.1  # deg/frame
extra_hold = fps * 33
zoom_in, zoom_out = fps*15, fps*5
max_zoom = 1.2

total_motion = len(frames_data)
total_frames = total_motion + extra_hold

#Compute bounds & centers

sample_idx = np.linspace(0, len(raw_frames)-1, min(len(raw_frames),10), dtype=int)
samples = np.vstack([raw_frames[i] for i in sample_idx])
bounds = [(samples[:,i].min()*1.0, samples[:,i].max()*1.0) for i in range(3)]
bounds = [(mn - 0.2*(mx-mn), mx + 0.2*(mx-mn)) for mn, mx in bounds]
centers = [0.5*(mn+mx) for mn, mx in bounds]

#Setup figure & axes

dpi = 300
fig = plt.figure(figsize=(6,6), dpi=dpi)
ax = fig.add_subplot(111, projection='3d')
ax.set_facecolor('black'); fig.patch.set_facecolor('black')
ax.axis('off'); ax.set_box_aspect([1,1,1])
elev0, azim0 = 25, 30
ax.view_init(elev0, azim0)
scat = ax.scatter([], [], [], s=0.5, alpha=0.1, c='cyan', edgecolors='none')
txt = ax.text2D(0.02, 0.95, '', transform=ax.transAxes, color='white')

#Inset for axes triad

ax_triad = inset_axes(ax, width="20%", height="20%", loc='lower left')
ax_triad.axis('off')

#Rotation Matrices

def rotation_matrix(ang_deg):
    theta = np.deg2rad(ang_deg)
    Rz = np.array([[np.cos(theta), -np.sin(theta), 0],
                   [np.sin(theta),  np.cos(theta), 0],
                   [0, 0, 1]])
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(theta), -np.sin(theta)],
                   [0, np.sin(theta),  np.cos(theta)]])
    return Rz @ Rx

#Update function

def update(idx):
    #data & redshift
    if idx < total_motion:
        data, zval = frames_data[idx], frames_z[idx]
    else:
        data, zval = frames_data[-1], frames_z[-1]
    scat._offsets3d = (data[:,0], data[:,1], data[:,2])
    txt.set_text(f'z = {zval:.3f}')
    #compute cumulative rotation angle
    angle = idx * rotation_speed
    #apply to main view
    ax.view_init(elev0 + angle, azim0 + angle)
    #inset triad
    ax_triad.clear(); ax_triad.axis('off')
    R = rotation_matrix(angle)
    origin = np.array([0,0,0])
    #unit axes
    axes = {'X':np.array([1,0,0]), 'Y':np.array([0,1,0]), 'Z':np.array([0,0,1])}
    for color, (label, vec) in zip(['red','green','blue'], axes.items()):
        vrot = R @ vec
        #project to 2D inset plane (ignore z)
        end = np.array([0.5,0.5]) + 0.3 * vrot[:2]
        ax_triad.annotate('', xy=end, xytext=(0.5,0.5),
                         arrowprops=dict(arrowstyle='->', color=color))
        ax_triad.text(end[0], end[1], label, color=color, fontsize=8)
    #zoom effect
    if idx >= total_motion:
        h = idx - total_motion
        if h <= zoom_in:
            f = 1 + (max_zoom-1)*(h/zoom_in)
        else:
            ho = h - zoom_in
            f = max_zoom - (max_zoom-1)*min(ho/zoom_out,1)
    else:
        f = 1   
    #adjust limits
    for i, (mn, mx) in enumerate(bounds):
        c = centers[i]; r = (mx-mn)/2
        low, high = c - r/f, c + r/f
        if i==0: ax.set_xlim(low, high)
        elif i==1: ax.set_ylim(low, high)
        else: ax.set_zlim(low, high)
    return scat, txt

#Animate & save

ani = FuncAnimation(fig, update, frames=total_frames, interval=1000/fps, blit=True)
writer = FFMpegWriter(fps=fps, metadata={'artist':'PragunNepal'}, bitrate=30000)
ani.save('N-body_Simulation.mp4', writer=writer)
print('✅ Saved N-body_Simulation.mp4')
```
---

## ⚙️ <a name="Code_for_N-body_Simulation_axes"></a>3. Code for N-body Simulation (with axes)

```python
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

#Extracting Redshift Value from FileName

def extract_redshift(fn):
    base = os.path.splitext(os.path.basename(fn))[0]
    parts = base.split("_")
    return float(parts[1] + "." + parts[2])

#Load & sort data

files = sorted(glob.glob("p_*.txt"), key=extract_redshift, reverse=True)

#If the files are not in the same directory, you can add its path
""" 
nbody_path = "your file path here"
nbody_files = sorted(
    glob.glob(os.path.join(nbody_path, "nbody_*.txt")),
    key=extract_redshift_halo,
    reverse=True 
"""

redshifts = [extract_redshift(f) for f in files]
raw_frames = [np.loadtxt(f).reshape(-1,3) for f in files]

#Interpolate data & redshifts

frames_data, frames_z = [], []
for (a, za), (b, zb) in zip(zip(raw_frames[:-1], redshifts[:-1]), zip(raw_frames[1:], redshifts[1:])):
    frames_data.append(a); frames_z.append(za)
    frames_data.append((a + b)/2); frames_z.append((za + zb)/2)
frames_data.append(raw_frames[-1]); frames_z.append(redshifts[-1])

#Animation parameters

fps = 30
rotation_speed = 0.1  # deg/frame
extra_hold = fps * 33
zoom_in, zoom_out = fps*15, fps*5
max_zoom = 1.2

total_motion = len(frames_data)
total_frames = total_motion + extra_hold

#Compute bounds & centers

sample_idx = np.linspace(0, len(raw_frames)-1, min(len(raw_frames),10), dtype=int)
samples = np.vstack([raw_frames[i] for i in sample_idx])
bounds = [(samples[:,i].min(), samples[:,i].max()) for i in range(3)]
bounds = [(mn - 0.2*(mx-mn), mx + 0.2*(mx-mn)) for mn, mx in bounds]
centers = [0.5*(mn+mx) for mn, mx in bounds]

#Setup figure & axes

dpi = 300
fig = plt.figure(figsize=(6,6), dpi=dpi)
ax = fig.add_subplot(111, projection='3d')

#background black

ax.set_facecolor('black'); fig.patch.set_facecolor('black')

#configure 3D grid on each axis with nearly invisible visibility

for axis in [ax.xaxis, ax.yaxis, ax.zaxis]:
    axis._axinfo['grid']['color'] = 'purple'
    axis._axinfo['grid']['linewidth'] = 0.1
    axis._axinfo['grid']['alpha'] = 0.001  # almost invisible

#axis panes extremely subtle purple

for axis in [ax.xaxis, ax.yaxis, ax.zaxis]:
    axis.pane.set_facecolor('purple')
    axis.pane.set_edgecolor('purple')
    axis.pane.set_alpha(0.001)  # almost invisible

#faint purple ticks

ax.tick_params(colors='purple', which='both', labelsize=6)
ax.set_box_aspect([1,1,1])
ax.axis('on')

#initial view & scatter

elev0, azim0 = 25, 30
ax.view_init(elev0, azim0)
scat = ax.scatter([], [], [], s=0.5, alpha=0.1, c='cyan', edgecolors='none')
txt = ax.text2D(0.02, 0.95, '', transform=ax.transAxes, color='white')
# inset for axes triad
ax_triad = inset_axes(ax, width="20%", height="20%", loc='lower left')
ax_triad.axis('off')

#rotation matrices

def rotation_matrix(ang_deg):
    theta = np.deg2rad(ang_deg)
    Rz = np.array([[np.cos(theta), -np.sin(theta), 0],
                   [np.sin(theta),  np.cos(theta), 0],
                   [0, 0, 1]])
    Rx = np.array([[1, 0, 0],
                   [0, np.cos(theta), -np.sin(theta)],
                   [0, np.sin(theta),  np.cos(theta)]])
    return Rz @ Rx

#Update function

def update(idx):
    #choose frame or hold
    if idx < total_motion:
        data, zval = frames_data[idx], frames_z[idx]
    else:
        data, zval = frames_data[-1], frames_z[-1]
    scat._offsets3d = (data[:,0], data[:,1], data[:,2])
    txt.set_text(f'z = {zval:.3f}')
    #rotate view
    angle = idx * rotation_speed
    ax.view_init(elev0 + angle, azim0 + angle)
    #zoom effect
    if idx >= total_motion:
        h = idx - total_motion
        if h <= zoom_in:
            f = 1 + (max_zoom-1)*(h/zoom_in)
        else:
            ho = h - zoom_in
            f = max_zoom - (max_zoom-1)*min(ho/zoom_out,1)
    else:
        f = 1
    #adjust limits for zoom
    for i, (mn, mx) in enumerate(bounds):
        c = centers[i]; r = (mx-mn)/2
        low, high = c - r/f, c + r/f
        if i==0: ax.set_xlim(low, high)
        elif i==1: ax.set_ylim(low, high)
        else: ax.set_zlim(low, high)
    #update inset triad
    ax_triad.clear(); ax_triad.axis('off')
    R = rotation_matrix(angle)
    for color, (label, vec) in zip(['red','green','blue'], {'X':np.array([1,0,0]), 'Y':np.array([0,1,0]), 'Z':np.array([0,0,1])}.items()):
        v2 = R @ vec
        end = np.array([0.5,0.5]) + 0.3 * v2[:2]
        ax_triad.annotate('', xy=end, xytext=(0.5,0.5), arrowprops=dict(arrowstyle='->', color=color, alpha=0.6))
        ax_triad.text(end[0], end[1], label, color=color, fontsize=8, alpha=0.6)
    return scat, txt

#Animate & save

ani = FuncAnimation(fig, update, frames=total_frames, interval=1000/fps, blit=True)
writer = FFMpegWriter(fps=fps, metadata={'artist':'PragunNepal'}, bitrate=30000)
ani.save('N-body_Simulation_with_axes.mp4', writer=writer)
print('✅ Saved N-body_Simulation_with_axes.mp4')
```
---

## 🔭 <a name="conclusion"></a>4. Conclusion

The code has been made general, hence its easily customisable. Please make necessary changes to the code as per your needs. 

©Pragun Nepal, 2025

Contact Me :
- Email : pragunnepal@gmail.com
- Email : pragun23@iisertvm.ac.in
- GitHub : PragunNepal
- LinkedIn : Pragun Nepal